In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

2023-10-16 13:19:37.030544: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

In [9]:
BATCH_SIZE = 128

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    # label = (example['age'] - 50) / 100
    label = example['age']
    return ecg_data[:,1], label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [10]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

In [13]:
def conv_block(x, n, k, mp):
    x = tf.keras.layers.Conv1D(n, k, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D()(x)

    return x

kernel_initializer = 'he_normal'
input_layer = tf.keras.layers.Input(shape=(5000))
x = input_layer
x = tf.keras.layers.Reshape((5000, 1))(x)

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [ ]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-16 13:29:04.702919: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7feb88ac5b00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-16 13:29:04.704493: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-10-16 13:29:04.840817: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-16 13:29:05.617953: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3904/3904 [==============================] - 312s 78ms/step - loss: 626.9379 - mse: 626.9379 - mae: 18.3059 - val_loss: 188.3338 - val_mse: 188.3338 - val_mae: 10.8005
Epoch 2/10
3904/3904 [==============================] - 264s 68ms/step - loss: 220.2998 - mse: 220.2998 - mae: 11.7982 - val_loss: 159.6990 - val_mse: 159.6990 - val_mae: 9.9987
Epoch 3/10
3904/3904 [==============================] - 265s 68ms/step - loss: 204.0125 - mse: 204.0125 - mae: 11.3689 - val_loss: 142.6672 - val_mse: 142.6672 - val_mae: 9.4516
Epoch 4/10
3904/3904 [==============================] - 265s 68ms/step - loss: 192.9807 - mse: 192.9807 - mae: 11.0467 - val_loss: 177.1965 - val_mse: 177.1965 - val_mae: 10.4363
Epoch 5/10
3904/3904 [==============================] - 264s 68ms/step - loss: 183.6500 - mse: 183.6500 - mae: 10.7720 - val_loss: 238.8665 - val_mse: 238.8665 - val_mae: 12.1500
Epoch 6/10
3904/3904 [==============================] - 264s 68ms/step - loss: 176.0502 - mse: 176.0502 - mae: 10.5475